### Блок тестовых вопросов

*Выберите верные утверждения про Lasso и Ridge*: 

1. Ridge-регуляризация более склонна занулять веса модели, нежели Lasso.
2. Lasso-регуляризация более склонна занулять веса модели, нежели Ridge.
3. Ridge и Lasso регуляризация призваны бороться с ситуацией недообучения. 
4. Суть Lasso регуляризации заключается в прибавлении к минимизируемому функционалу суммы модулей обучаемых коэффициентов.
5. Суть Lasso регуляризации заключается в прибавлении к минимизируемому функционалу суммы квадратов обучаемых коэффициентов.



**Ответ: 2, 4)** Оба вида регуляризации, как было показано в лекции, можно свести к поиску условного экстремума на некотором компактном множестве. В случае Ridge регуляризации данное множество оказывается выпуклым (например, в двухмерном пространстве – это круг), ровно как минимизируемый функционал. Тогда шанс того, что линия уровня и граница множества из ограничения соприкоснутся где-то на оси, достаточно маленький. В случае Lasso ситуация обратная (см. Визуальное объяснение в лекции). 
_______________________________________________________

*Выберите верные утверждения про Мультиколлинеарность*: 

1. Мультиколлинеарность гарантирует, что мы получим переобученную модель.
2. Мультиколлинеарность возникает при сильной линейной зависимости в матрице объект-признак.
3. Если в матрице объект-признак присутствуют линейно зависимые признаки, тогда с вероятностью 50% матричную формулу поиска оптимальных коэффициентов регрессии применить нельзя.
4. Если в матрице объект-признак присутствуют линейно зависимые признаки, тогда у минимизируемого функционала оказывается не одна единственная, а уже 2-3 точки минимума.
5. Если в матрице объект-признак присутствуют линейно зависимые признаки, тогда у минимизируемого функционала оказывается бесконечное количество минимумов.
6. Мультиколлинеарность можно полечить, удалив зависимые признаки или используя регуляризацию.




**Ответ: 2, 5, 6)** важно понимать, что данные рассуждения относятся именно к задаче построения линейной регрессии прям как в лекции назад. Для других моделей мультиколлинеарность может, вообще говоря, ни к каким проблемам не приводить.
_______________________________________________________

*Выберите верные утверждения про масштабирование признаков:* 

1. Масштабировать признаки необходимо всегда. Иначе качество на трейне может сильно превышать качество на тесте.
2. Масштабировать признаки рекомендуется всегда. Особенно в задачах, где используется метод градиентного спуска: он становится быстрее.
3. Мы масштабируем признаки, регуляризируя модели, чтобы с одинаковой степенью справедливости учитывать порядок фичей, и, соответственно, чтобы случайно не штрафовать сильно больше те коэффициенты, которые по природе своей должны быть на порядок выше остальных.
4. Подход StandardScaler масштабирует признаки таким образом, что все значения переводятся в промежуток [0; 1].
5. Подход MinMaxScaler масштабирует признаки таким образом, что все значения переводятся в промежуток [0; 1].



**Ответ: 2, 3, 5)** знак производной (положительная она или отрицательная) поясняет нам: возрастает функция или убывает. Минимумы у функции возникают там, где она какое-то время убывала, а потом в моменте стала возрастать. Поэтому такой переход для искомых точек подходит под 1 вариант ответа.

_______________________________________________________

### Блок практики

### Задание 4

### Замерим качество Линейной регрессии после обработки данных на Кросс-Валидации на 4 фолдах! (в прошлый раз делали 20 фолдов)

In [6]:
import numpy as np
import pandas as pd

processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [7]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


In [8]:
from sklearn.model_selection import KFold

selector = KFold(n_splits=4, shuffle=True, random_state=33)

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

### Построим обыкновенную модель линейной регрессии
### И замерим качество на установленной кросс-валидации

model = LinearRegression()

X = processed_data.drop('log_trip_duration', axis=1)
Y = processed_data['log_trip_duration']

cv_result_1 = cross_validate(model, X, Y, 
                             scoring='neg_mean_squared_error',
                             cv=selector, return_train_score=True)

cross_val_error = -np.mean(cv_result_1['test_score'])

print(f"RMSLE на Кросс-валидации: {round(cross_val_error, 3)}")

RMSLE на Кросс-валидации: 0.425


### Задание 5

## Поработал один из хитрых гномов!

В отличие от своих собратьев, третий гном оказался тем еще бездельником в школьные годы, но все равно страстно желал во всем догнать первых двух. И сейчас, желая помочь им в построении модели по предсказанию длительности поездки такси, добавил в данные 20 зашифрованных фичей (их смысл нам не рассказали: какая-то секретная информация о водителях).

Гном думал следующим образом: "Ну не может же модель стать хуже! А тут вот авось и мое нововведение уменьшит ошибку в разы! Тогда и меня станут звать на гномий  data-саммит."

Проверим на кросс-валидации, насколько гном оказался прав!

In [13]:
### Загрузим новый гномий датасет

new_data = pd.read_csv('new_data.csv', index_col='id')

In [14]:
new_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,1,1,1,1,1,...,0,0,0,0,0,1.500479,2.251437,3.378234,5.068969,7.605881
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0,0,0,...,0,0,0,0,0,1.807119,3.265681,5.901475,10.664670,19.272331
id3858529,1,930.399753,0,6.392080,7.661527,1,1,1,1,1,...,0,0,0,0,0,6.392080,40.858690,261.172025,1669.432545,10671.146803
id3504673,1,930.399753,0,1.487155,6.063785,1,1,1,1,1,...,0,0,0,0,0,1.487155,2.211629,3.289035,4.891303,7.274125
id2181028,1,930.399753,0,1.189925,6.077642,1,1,1,1,1,...,0,0,0,0,0,1.189925,1.415923,1.684842,2.004837,2.385606


In [15]:
### Замерим теперь качество с новыми фичами

X_2 = new_data.drop('log_trip_duration', axis=1)
Y_2 = new_data['log_trip_duration']

cv_result_2 = cross_validate(model, X_2, Y_2, 
                             scoring='neg_mean_squared_error',
                             cv=selector, return_train_score=True)

In [16]:
cross_val_error_2 = -np.mean(cv_result_2['test_score'])

print(f"RMSLE на Кросс-валидации: {round(cross_val_error_2, 3)}")

RMSLE на Кросс-валидации: 144.716


### Задание 6

В линейной алгербре зачастую используют понятие **ранга матрицы**. Оно соответствует кол-ву линейно независимых столбцов в матрице. Иными словами, позволяет оценить, есть ли избыток информации в нашем датафрейме. Если ранг матрицы меньше, чем кол-во используемых столбцов, то некоторые фичи следует удалить, ведь иначе возникате ситуация строгой мультиколлинеарности.

Чтобы замерить ранг в наших матрицах объект-признак, можно воспользоваться функцией numpy.linalg.matrix_rank

Константным признаком в данном упражнении можно пренебречь.

In [17]:
print(f"В первой модели всего фичей: {X.shape[1]}, - а ранг равен {np.linalg.matrix_rank(X)}")

print(f"В первой модели всего фичей: {X_2.shape[1]}, - а ранг равен {np.linalg.matrix_rank(X_2)}")

В первой модели всего фичей: 4, - а ранг равен 4
В первой модели всего фичей: 24, - а ранг равен 5


### Задания 7, 8

Не кажется ли нам, что из-за новых 20 фичей появилась проблема мультиколлинеарности? Как поступить гному, чтобы, с одной стороны, получить адекватное качество, а с другой стороны, не повредить свое самолюбие и не убирать новые признаки?

Верно! Например, с помощью регуляризации.

Найдите такой параметр регуляризации $\lambda$ для Ridge и Lasso случая, чтобы ошибка RMSLE на кросс-валидации оказалась строго меньше 0.4

**ALARM**: используйте процедуру масшратибрования данных (воспользуйтесь методом MinMaxScaler) перед тем как применить регуляризацию. Важно - чтобы сохранить концепцию независимости обучения на трейне и на тесте, на каждой итерации кросс-валидации необходимо замерять параметры стандартизации исключительно на трейне, а потом применять на валидационном фолде.

In [19]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

from sklearn.linear_model import Lasso, Ridge

from sklearn.model_selection import GridSearchCV

In [20]:
### Добавить масштабирование как шаг в Pipeline и обучим lasso

pipe_lasso = Pipeline([('scaler', MinMaxScaler()),
                       ('lasso', Lasso(max_iter=100000))])

alphas = np.linspace(0.00001, 0.0001, num=30)

### Переберем кучу степеней регуляризации
### Заметим, что найдутся лучшие, у которых качество лучше,
### Чем требуемое

cv_lasso = GridSearchCV(pipe_lasso, param_grid={'lasso__alpha':alphas},
                        scoring='neg_mean_squared_error', cv=selector,
                        return_train_score=True)

cv_lasso.fit(X_2, Y_2)

GridSearchCV(cv=KFold(n_splits=4, random_state=33, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('lasso', Lasso(max_iter=100000))]),
             param_grid={'lasso__alpha': array([1.00000000e-05, 1.31034483e-05, 1.62068966e-05, 1.93103448e-05,
       2.24137931e-05, 2.55172414e-05, 2.86206897e-05, 3.17241379e-05,
       3.48275862e-05, 3.79310345e-05, 4.10344828e-05, 4.41379310e-05,
       4.72413793e-05, 5.03448276e-05, 5.34482759e-05, 5.65517241e-05,
       5.96551724e-05, 6.27586207e-05, 6.58620690e-05, 6.89655172e-05,
       7.20689655e-05, 7.51724138e-05, 7.82758621e-05, 8.13793103e-05,
       8.44827586e-05, 8.75862069e-05, 9.06896552e-05, 9.37931034e-05,
       9.68965517e-05, 1.00000000e-04])},
             return_train_score=True, scoring='neg_mean_squared_error')

In [21]:
### Самая лучшая модель на Кросс-Валидации

cv_lasso.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('lasso', Lasso(alpha=1.310344827586207e-05, max_iter=100000))])

In [22]:
### Список различных скоров для всех моделей
### Которые были перебраны выше

cv_lasso.cv_results_['mean_test_score']

array([-0.39997066, -0.3905583 , -0.39340447, -0.40079674, -0.40039685,
       -0.40003784, -0.39972019, -0.39944458, -0.39920975, -0.39901531,
       -0.3988616 , -0.39874874, -0.39867429, -0.3986393 , -0.39864062,
       -0.39867251, -0.39873549, -0.39882859, -0.3989591 , -0.39912467,
       -0.39932886, -0.39957195, -0.39985352, -0.400174  , -0.40053304,
       -0.40093091, -0.40136755, -0.4018427 , -0.40235664, -0.40290931])

In [23]:
### Ridge

pipe_ridge = Pipeline([('scaler', MinMaxScaler()), ('ridge', Ridge())])

cv_ridge = GridSearchCV(pipe_ridge, param_grid={'ridge__alpha':alphas},
                        scoring='neg_mean_squared_error', cv=selector,
                        return_train_score=True)

cv_ridge.fit(X_2, Y_2)

GridSearchCV(cv=KFold(n_splits=4, random_state=33, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-05, 1.31034483e-05, 1.62068966e-05, 1.93103448e-05,
       2.24137931e-05, 2.55172414e-05, 2.86206897e-05, 3.17241379e-05,
       3.48275862e-05, 3.79310345e-05, 4.10344828e-05, 4.41379310e-05,
       4.72413793e-05, 5.03448276e-05, 5.34482759e-05, 5.65517241e-05,
       5.96551724e-05, 6.27586207e-05, 6.58620690e-05, 6.89655172e-05,
       7.20689655e-05, 7.51724138e-05, 7.82758621e-05, 8.13793103e-05,
       8.44827586e-05, 8.75862069e-05, 9.06896552e-05, 9.37931034e-05,
       9.68965517e-05, 1.00000000e-04])},
             return_train_score=True, scoring='neg_mean_squared_error')

In [24]:
cv_ridge.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('ridge', Ridge(alpha=5.9655172413793106e-05))])

In [25]:
cv_lasso.cv_results_['mean_test_score']

array([-0.39997066, -0.3905583 , -0.39340447, -0.40079674, -0.40039685,
       -0.40003784, -0.39972019, -0.39944458, -0.39920975, -0.39901531,
       -0.3988616 , -0.39874874, -0.39867429, -0.3986393 , -0.39864062,
       -0.39867251, -0.39873549, -0.39882859, -0.3989591 , -0.39912467,
       -0.39932886, -0.39957195, -0.39985352, -0.400174  , -0.40053304,
       -0.40093091, -0.40136755, -0.4018427 , -0.40235664, -0.40290931])